# 模型适应性测试

## 导入依赖库，传入模型参数

In [ ]:
import os
import random
from datetime import datetime

import numpy as np
import torch

class Args:
    def __init__(self, **kwargs):
        for key, value in kwargs.items():
            setattr(self, key, value)

args = Args(
    task_name="long_term_forecast",
    is_training=1,
    root_path="./dataset/Gas/",
    data_path="134312_data.csv",
    model_id="Gas_96_96",
    model="DLinear",
    data="Gas",
    features="M",
    target="value",
    seq_len=96,
    label_len=48,
    pred_len=96,
    e_layers=2,
    d_layers=1,
    d_model=512,
    n_heads=8,
    factor=3,
    enc_in=8,
    dec_in=8,
    c_out=8,
    des="Exp",
    itr=1,
    use_gpu=True,
    gpu="0",
    device="2,3",
    loss="MSE",
    distil=True,
    embed="timeF",
    seed=0,
    local_rank=0,
    d_ff=2048,
    adaptation=False,
    noiseness=False,
    periodicity=False,
    distribution=False,
    anomaly=False,
    moving_avg=25,
    freq="h",
    checkpoints="./checkpoints/",
    feature="M",
    inverse=False,
    seasonal_patterns="Monthly",
    num_workers=10,
    batch_size=32,
    use_ims=False,
    use_weight_decay=0,
    learning_rate=0.0001,
    use_amp=False,
    output_attention=None,
)

args.output_len = args.pred_len
args.output_len_list = [args.output_len]

fix_seed = args.seed
random.seed(fix_seed)
torch.manual_seed(fix_seed)
np.random.seed(fix_seed)
args.use_gpu = True if torch.cuda.is_available() and args.use_gpu else False

ii = 0
setting = '{}_{}_{}_{}_ft{}_sl{}_ll{}_pl{}_dm{}_nh{}_el{}_dl{}_df{}_fc{}_eb{}_dt{}_{}'.format(
    args.task_name,
    args.model_id,
    args.model,
    args.data,
    args.features,
    args.seq_len,
    args.label_len,
    args.pred_len,
    args.d_model,
    args.n_heads,
    args.e_layers,
    args.d_layers,
    args.d_ff,
    args.factor,
    args.embed,
    args.distil,
    args.des,
    ii)

## 噪声适应性
* `add_gaussian_white_noise`：添加高斯白噪声
* `add_red_noise`：添加红噪声

In [2]:
def add_gaussian_white_noise(tensor, mean=0.0, std=1.0, rd=1):
    noise = torch.normal(mean, std * rd, tensor.size())
    return tensor + noise

def add_red_noise(tensor, alpha=0.5, rd=1):
    alpha = alpha * rd
    batch_size, seq_len, feature_num = tensor.size()
    red_noise = torch.zeros_like(tensor)
    
    red_noise[:, 0, :] = torch.normal(0, 1, (batch_size, feature_num))
    
    for i in range(1, seq_len):
        red_noise[:, i, :] = alpha * red_noise[:, i-1, :] + torch.normal(0, (1 - alpha**2)**0.5, (batch_size, feature_num))
    
    return tensor + red_noise

## 周期性适应性
* `phase_shift`：添加相位偏移
* `period_scale`：添加周期性缩放

In [3]:
def phase_shift(tensor, shift=None, rd=1):
    batchsize, seqlen, featurenum = tensor.shape
    shift = int(rd * seqlen * 0.2)
    if shift >= 0:
        shifted_tensor = torch.cat([tensor[:, shift:, :], tensor[:, :shift, :]], dim=1)
    else:
        shift = abs(shift)
        shifted_tensor = torch.cat([tensor[:, seqlen-shift:, :], tensor[:, :seqlen-shift, :]], dim=1)
    return shifted_tensor

def period_scale(tensor, scale=2.0, rd=1):
    batchsize, seqlen, featurenum = tensor.shape

    new_seqlen = int(seqlen * scale * rd)  # 使用 rd 调整缩放比例
    
    # 创建时间轴的坐标范围为 0 到 1，以保持周期性
    time_axis = torch.linspace(0, 1, steps=seqlen, device=tensor.device, dtype=tensor.dtype)
    new_time_axis = torch.linspace(0, 1, steps=new_seqlen, device=tensor.device, dtype=tensor.dtype)
    
    # 为每个特征创建网格
    y_axis = torch.arange(featurenum, device=tensor.device, dtype=tensor.dtype)
    grid = torch.stack(torch.meshgrid(time_axis, y_axis, indexing='ij'), dim=-1)  # (seqlen, featurenum, 2)
    grid = grid.unsqueeze(0).repeat(batchsize, 1, 1, 1)  # (batchsize, seqlen, featurenum, 2)
    
    # 调整网格到新的序列长度
    new_grid = torch.stack(torch.meshgrid(new_time_axis, y_axis, indexing='ij'), dim=-1)  # (new_seqlen, featurenum, 2)
    new_grid = new_grid.unsqueeze(0).repeat(batchsize, 1, 1, 1)  # (batchsize, new_seqlen, featurenum, 2)
    
    # 使用 grid_sample 进行采样
    tensor = tensor.unsqueeze(1)  # (batchsize, 1, seqlen, featurenum)
    tensor = torch.nn.functional.grid_sample(tensor, new_grid, mode='bilinear', align_corners=True).squeeze(1)  # (batchsize, new_seqlen, featurenum)
    
    if new_seqlen > seqlen:
        tensor = tensor[:, :seqlen, :]
    else:
        pad_size = seqlen - new_seqlen
        tensor = torch.cat([tensor, tensor[:, :pad_size, :]], dim=1)

    return tensor

## 数据分布适应性
* `global_mean_shift`：添加全局均值偏移
* `data_range_scaling`：添加数据范围缩放

In [4]:
def global_mean_shift(tensor, shift_amount=1, rd=1):
    if isinstance(shift_amount, (int, float)):
        shift_amount = tensor.new_full((1, 1, tensor.size(2)), shift_amount)
    elif isinstance(shift_amount, torch.Tensor):
        assert shift_amount.size(0) == tensor.size(2), "the length of shift_amount must be the same as feature_num"
    else:
        raise ValueError("shift_amount must be a scalar or a vector of the same length as feature_num")
    
    return tensor + shift_amount * rd

def data_range_scaling(tensor, scale_factor=0.9, rd=1):
    if isinstance(scale_factor, (int, float)):
        scale_factor = tensor.new_full((1, 1, tensor.size(2)), scale_factor)
    elif isinstance(scale_factor, torch.Tensor):
        assert scale_factor.size(0) == tensor.size(2), "the length of scale_factor must be the same as feature_num"
    else:
        raise ValueError("scale_factor must be a scalar or a vector of the same length as feature_num")
    
    return tensor * scale_factor * rd

## 数据异常适应性
* `add_missing_values`：添加缺失值
* `add_outlier_values`：添加异常极大的值

In [5]:
def add_missing_values(tensor, missing_rate=0.05, rd=1):
    mask = torch.rand(tensor.size()) < missing_rate * rd
    tensor[mask] = 0.0
    return tensor

def add_outlier_values(tensor, outlier_rate=0.05, magnitude=20, rd=1):
    mask = torch.rand(tensor.size()) < outlier_rate * rd
    # 生成异常值，并确保异常值的类型与 tensor 相同
    outliers = magnitude * torch.randn(*tensor.size()).to(tensor.dtype)
    tensor[mask] = outliers[mask]
    return tensor

## 根据不同任务选择对应的适应性函数

In [6]:
def adaptivity(x, i, args, rd=1):
    adp_type = ""
    if i % 2 == 0:
        if args.noiseness:
            trans_x = add_gaussian_white_noise(x, rd)
            adp_type = "noiseness"
        if args.periodicity:
            trans_x = phase_shift(x, rd)
            adp_type = "periodicity"
        if args.distribution:
            trans_x = global_mean_shift(x, rd)
            adp_type = "distribution"
        if args.anomaly:
            trans_x = add_missing_values(x, rd)
            adp_type = "anomaly"
    else:
        if args.noiseness:
            trans_x = add_red_noise(x, rd)
            adp_type = "noiseness"
        if args.periodicity:
            trans_x = period_scale(x, rd)
            adp_type = "periodicity"
        if args.distribution:
            trans_x = data_range_scaling(x, rd)
            adp_type = "distribution"
        if args.anomaly:
            trans_x = add_outlier_values(x, rd)
            adp_type = "anomaly"
    return trans_x, adp_type

## 推理部分
* **适应性指标计算**(`adaptation_test`)：将原测试集数据替换为适应性变换后数据，输入给模型进行推理，预测结果与ground truth比较，计算MSE的平均值，得到各项适应性指标。指标结果越小，表明适应性越好。
* **适应性指标验证**(`adaptation_varify`)：模拟真实数据使用环境，将原测试集数据添加随机强度的适应性变换，得到模拟真实测试集，输入给模型进行推理，预测结果与ground truth比较，计算MSE的平均值，作为适应性指标验证结果。适应性指标与真实适应性指标验证结果呈正相关，适应性指标验证成功。

In [7]:
from utils.tools import *
from data_provider.data_factory import data_provider
from utils.metrics import metric
import torch.nn as nn
from torch import optim
from models import DLinear

class Exp(object):
    def __init__(self, args):
        self.args = args
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model = self._build_model().to(self.device)
        
    def _build_model(self):
        model = DLinear.Model(self.args)
        return model

    def _get_data(self, flag):
        data_set, data_loader = data_provider(self.args, flag)
        return data_set, data_loader

    def _select_optimizer(self):
        if self.args.use_weight_decay:
            model_optim = optim.Adam(self.model.parameters(), lr=self.args.learning_rate,
                                     weight_decay=self.args.weight_decay)
        else:
            model_optim = optim.Adam(self.model.parameters(), lr=self.args.learning_rate)
        return model_optim

    def _select_criterion(self):
        criterion = nn.MSELoss()
        return criterion

    # 适应性测试，计算适应性指标
    def adaptation_test(self, setting):
        print('Model parameters: ', sum(param.numel() for param in self.model.parameters()))
        attns = []
        folder_path = './test_results/' + setting + '/' + self.args.data_path + '/' + f'{self.args.output_len}/'
        if not os.path.exists(folder_path) and int(os.environ.get("LOCAL_RANK", "0")) == 0:
            os.makedirs(folder_path)
        self.model.eval()
        if self.args.output_len_list is None:
            self.args.output_len_list = [self.args.output_len]

        trues_list = [[] for _ in range(len(self.args.output_len_list))]
        trans_list = [[] for _ in range(len(self.args.output_len_list))]
        self.args.output_len_list.sort()
        adp_type = ""
        
        # pic_path = './adapt/' + setting + '/' + self.args.data_path + '/'

        with torch.no_grad():
            for output_ptr in range(len(self.args.output_len_list)):
                self.args.output_len = self.args.output_len_list[output_ptr]
                test_data, test_loader = data_provider(self.args, flag='test')
                
                for i, (batch_x, batch_y, batch_x_mark, batch_y_mark) in enumerate(test_loader):
                    # 适应性数据替换
                    if i % 2 == 0:
                        trans_batch_x, adp_type = adaptivity(batch_x, i, self.args, rd=1)
                    else:
                        trans_batch_x, adp_type = adaptivity(batch_x, i, self.args, rd=1)
                    # 推理
                    trans_pred, true = self.adaptation_forecast(test_data, trans_batch_x, batch_y, batch_x_mark, batch_y_mark)
                
                    trues_list[output_ptr].append(true)
                    trans_list[output_ptr].append(trans_pred)

        if self.args.output_len_list is not None:
            for i in range(len(trues_list)):
                true = trues_list[i]
                trans_preds = trans_list[i]
                true = torch.cat(true, dim=0).numpy()
                trans_preds = torch.cat(trans_preds, dim=0).numpy()
                
                mae, mse, rmse, mape, mspe = metric(true, trans_preds)
                print(f"output_len: {self.args.output_len_list[i]}")
                print('{} adaptation:{}'.format(adp_type, mae))
        
        return

    def adaptation_forecast(self, test_data, batch_x, batch_y, batch_x_mark, batch_y_mark):
        batch_x = batch_x.float().to(self.device)
        batch_y = batch_y.float().to(self.device)
        batch_x_mark = batch_x_mark.float().to(self.device)
        batch_y_mark = batch_y_mark.float().to(self.device)

        dec_inp = torch.zeros_like(batch_y[:, -self.args.pred_len:, :]).float()
        dec_inp = torch.cat([batch_y[:, :self.args.label_len, :], dec_inp], dim=1).float().to(self.device)
        inference_steps = self.args.output_len // self.args.pred_len
        dis = self.args.output_len - inference_steps * self.args.pred_len
        if dis != 0:
            inference_steps += 1
        pred_y = []
        for j in range(inference_steps):
            if len(pred_y) != 0:
                batch_x = torch.cat([batch_x[:, self.args.pred_len:, :], pred_y[-1]], dim=1)
                tmp = batch_y_mark[:, j - 1:j, :]
                batch_x_mark = torch.cat([batch_x_mark[:, 1:, :], tmp], dim=1)

            outputs = self.model(batch_x, batch_x_mark, dec_inp, batch_y_mark)

            f_dim = -1 if self.args.features == 'MS' else 0
            pred_y.append(outputs[:, -self.args.pred_len:, :])
        pred_y = torch.cat(pred_y, dim=1)

        if dis != 0:
            pred_y = pred_y[:, :-dis, :]

        if self.args.use_ims:
            batch_y = batch_y[:, self.args.label_len:self.args.label_len + self.args.output_len, :].to(
                self.device)
        else:
            batch_y = batch_y[:, :self.args.output_len, :].to(self.device)

        outputs = pred_y.detach().cpu()
        batch_y = batch_y.detach().cpu()
            
        if test_data.scale and self.args.inverse:
            shape = outputs.shape
            outputs = test_data.inverse_transform(outputs.squeeze(0)).reshape(shape)
            batch_y = test_data.inverse_transform(batch_y.squeeze(0)).reshape(shape)

        outputs = outputs[:, :, f_dim:]
        batch_y = batch_y[:, :, f_dim:]

        pred = outputs
        true = batch_y
        
        return pred, true

    # 验证适应性指标
    def adaptation_varify(self, setting):
        attns = []
        folder_path = './test_results/' + setting + '/' + self.args.data_path + '/' + f'{self.args.output_len}/'
        if not os.path.exists(folder_path) and int(os.environ.get("LOCAL_RANK", "0")) == 0:
            os.makedirs(folder_path)
        self.model.eval()
        if self.args.output_len_list is None:
            self.args.output_len_list = [self.args.output_len]

        trues_list = [[] for _ in range(len(self.args.output_len_list))]
        trans_list = [[] for _ in range(len(self.args.output_len_list))]
        self.args.output_len_list.sort()
        adp_type = ""
        
        # pic_path = './adapt/' + setting + '/' + self.args.data_path + '/'

        with torch.no_grad():
            for output_ptr in range(len(self.args.output_len_list)):
                self.args.output_len = self.args.output_len_list[output_ptr]
                test_data, test_loader = data_provider(self.args, flag='test')
                
                for i, (batch_x, batch_y, batch_x_mark, batch_y_mark) in enumerate(test_loader):
                    if i % 50 == 0 or i % 51 == 0:
                        # 生成系统随机数
                        random_bytes = os.urandom(4)
                        random_int = int.from_bytes(random_bytes, 'big')
                        random_float = (random_int >> 11) / (1 << 23) + 0.5
                        # 适应性数据替换
                        trans_batch_x, adp_type = adaptivity(batch_x, i, self.args, rd=random_float)
                    # 推理
                    trans_pred, true = self.adaptation_forecast(test_data, trans_batch_x, batch_y, batch_x_mark, batch_y_mark)
                    
                    trues_list[output_ptr].append(true)
                    trans_list[output_ptr].append(trans_pred)

        if self.args.output_len_list is not None:
            for i in range(len(trues_list)):
                true = trues_list[i]
                trans_preds = trans_list[i]
                true = torch.cat(true, dim=0).numpy()
                trans_preds = torch.cat(trans_preds, dim=0).numpy()
                
                mae, mse, rmse, mape, mspe = metric(true, trans_preds)
                print('{} adaptation verify:{}'.format(adp_type, mae))
        
        return
    
    def fgsm_attack(self, data, epsilon, data_grad):
        # Collect the element-wise sign of the data gradient
        sign_data_grad = data_grad.sign()
        # Create the perturbed image by adjusting each pixel of the input image
        perturbed_data = data + epsilon*sign_data_grad
        # Adding clipping to maintain [0,1] range
        # perturbed_data = torch.clamp(perturbed_data, 0, 1)
        # Return the perturbed image
        return perturbed_data

    def adversarial_attack(self, setting, test=0):
        test_data, test_loader = self._get_data(flag='test')
        if test:
            if self.args.ckpt_path != '':
                if self.args.ckpt_path == 'random':
                    print('loading model randomly')
                else:
                    print('loading model: ', self.args.ckpt_path)
                    if self.args.ckpt_path.endswith('.pth'):
                        self.model.load_state_dict(torch.load(self.args.ckpt_path))
                    else:
                        raise NotImplementedError
            else:
                print('loading model with settings: {}'.format(setting))
                self.model.load_state_dict(torch.load(os.path.join('./checkpoints/' + setting, 'checkpoint.pth')))
            
        preds = []
        trues = []
        adv_preds = []
        folder_path = './test_results/' + setting + '/'
        if not os.path.exists(folder_path):
            os.makedirs(folder_path)

        self.model.eval()
        model_optim = self._select_optimizer()
        criterion = self._select_criterion()
        
        for i, (batch_x, batch_y, batch_x_mark, batch_y_mark) in enumerate(test_loader):
            model_optim.zero_grad()
            batch_x = batch_x.float().to(self.device)
            batch_y = batch_y.float().to(self.device)
            batch_x.requires_grad= True

            batch_x_mark = batch_x_mark.float().to(self.device)
            batch_y_mark = batch_y_mark.float().to(self.device)

            # decoder input
            dec_inp = torch.zeros_like(batch_y[:, -self.args.pred_len:, :]).float()
            dec_inp = torch.cat([batch_y[:, :self.args.label_len, :], dec_inp], dim=1).float().to(self.device)
            # encoder - decoder
            if self.args.use_amp:
                with torch.cuda.amp.autocast():
                    if self.args.output_attention:
                        outputs = self.model(batch_x, batch_x_mark, dec_inp, batch_y_mark)[0]
                    else:
                        outputs = self.model(batch_x, batch_x_mark, dec_inp, batch_y_mark)
            else:
                if self.args.output_attention:
                    outputs = self.model(batch_x, batch_x_mark, dec_inp, batch_y_mark)[0]
                else:
                    outputs = self.model(batch_x, batch_x_mark, dec_inp, batch_y_mark)

            f_dim = -1 if self.args.features == 'MS' else 0
            outputs = outputs[:, -self.args.pred_len:, :]
            batch_y = batch_y[:, -self.args.pred_len:, :].to(self.device)
            loss = criterion(outputs,batch_y)
            loss.backward()
            data_grad = batch_x.grad.data
            if i % 2 == 0:
                adv_batch_x = self.fgsm_attack(batch_x, 0.1, data_grad)
            else:
                adv_batch_x = self.fgsm_attack(batch_x, 0.07, data_grad)

            if self.args.use_amp:
                with torch.cuda.amp.autocast():
                    if self.args.output_attention:
                        adv_outputs = self.model(adv_batch_x, batch_x_mark, dec_inp, batch_y_mark)[0]
                    else:
                        adv_outputs = self.model(adv_batch_x, batch_x_mark, dec_inp, batch_y_mark)
            else:
                if self.args.output_attention:
                    adv_outputs = self.model(adv_batch_x, batch_x_mark, dec_inp, batch_y_mark)[0]
                else:
                    adv_outputs = self.model(adv_batch_x, batch_x_mark, dec_inp, batch_y_mark)

            f_dim = -1 if self.args.features == 'MS' else 0
            adv_outputs = adv_outputs[:, -self.args.pred_len:, :]

            outputs = outputs.detach().cpu().numpy()
            adv_outputs = adv_outputs.detach().cpu().numpy()
            batch_y = batch_y.detach().cpu().numpy()

            if ',' in self.args.data:
                if test_data.datasets[0].scale and self.args.inverse:
                    shape = outputs.shape
                    outputs = test_data.inverse_transform(outputs.squeeze(0)).reshape(shape)
                    batch_y = test_data.inverse_transform(batch_y.squeeze(0)).reshape(shape)
            else:
                if test_data.scale and self.args.inverse:
                    shape = outputs.shape
                    outputs = test_data.inverse_transform(outputs.reshape(shape[0] * shape[1], -1)).reshape(shape)
                    batch_y = test_data.inverse_transform(batch_y.reshape(shape[0] * shape[1], -1)).reshape(shape)
                    adv_outputs = test_data.inverse_transform(adv_outputs.reshape(shape[0] * shape[1], -1)).reshape(shape)

            outputs = outputs[:, :, f_dim:]
            batch_y = batch_y[:, :, f_dim:]
            adv_outputs = adv_outputs[:, :, f_dim:]

            pred = outputs
            true = batch_y
            adv_pred = adv_outputs

            preds.append(pred)
            trues.append(true)
            adv_preds.append(adv_pred)

            if i % 5 == 0:
                input = batch_x.detach().cpu().numpy()
                adv_input = adv_batch_x.detach().cpu().numpy()
                if ',' in self.args.data:
                    if test_data.datasets[0].scale and self.args.inverse:
                        shape = input.shape
                        input = test_data.inverse_transform(input.squeeze(0)).reshape(shape)
                else:
                    if test_data.scale and self.args.inverse:
                        shape = input.shape
                        input = test_data.inverse_transform(input.reshape(shape[0] * shape[1], -1)).reshape(shape)
                        adv_input = test_data.inverse_transform(adv_input.reshape(shape[0] * shape[1], -1)).reshape(shape)
                # gt = np.concatenate((input[0, :, -1], true[0, :, -1]), axis=0)
                # pd = np.concatenate((input[0, :, -1], pred[0, :, -1]), axis=0)
                # adv = np.concatenate((adv_input[0, :, -1], adv_pred[0, :, -1]), axis=0)
                # visual(gt, pd, adv, os.path.join(folder_path, str(i) + '.pdf'))

        preds = np.array(preds)
        trues = np.array(trues)
        adv_preds = np.array(adv_preds)
        # print('test shape:', preds.shape, trues.shape, adv_preds.shape)
        preds = preds.reshape(-1, preds.shape[-2], preds.shape[-1])
        trues = trues.reshape(-1, trues.shape[-2], trues.shape[-1])
        adv_preds = adv_preds.reshape(-1, adv_preds.shape[-2], adv_preds.shape[-1])
        # print('test shape:', preds.shape, trues.shape, adv_preds.shape)

        # result save
        folder_path = './results/' + setting + '/'
        if not os.path.exists(folder_path):
            os.makedirs(folder_path)

        mae, mse, rmse, mape, mspe = metric(preds, trues)
        adv_mae, adv_mse, adv_rmse, adv_mape, adv_mspe = metric(adv_preds, trues)
        print(f"output_len: {self.args.output_len}")
        print('Before attacking: mse:{}, mae:{}'.format(mse, mae))
        print('After attacking: mse:{}, mae:{}'.format(adv_mse, adv_mae))
        print("Adversary adaptation:{}".format(adv_mae))

        f = open("result_adaptation_test_forecast.txt", 'a')
        f.write(setting + "  \n")
        f.write('Before attacking: mse:{}, mae:{}'.format(mse, mae))
        f.write('\n')
        f.write('After attacking: mse:{}, mae:{}'.format(adv_mse, adv_mae))
        f.write('\n')
        f.write('\n')
        f.close()

        # np.save(folder_path + 'metrics.npy', np.array([mae, mse, rmse, mape, mspe]))
        # np.save(folder_path + 'pred.npy', preds)
        # np.save(folder_path + 'true.npy', trues)

        return

### 噪声适应性指标计算及验证
* 调用适应性指标计算接口`adaptation_test`，计算指标
* 调用适应性指标验证接口`adaptation_varify`，验证指标

In [8]:
args.adaptation = True
args.noiseness = True
exp = Exp(args)
exp.adaptation_test(setting)
exp.adaptation_varify(setting)

Model parameters:  18624
test 1905
output_len: 96
noiseness adaptation:1.177431583404541
test 1905
noiseness adaptation verify:1.008675217628479


### 周期性适应性指标计算及验证

In [9]:
args.adaptation = True
args.noiseness = False
args.periodicity = True
exp = Exp(args)
exp.adaptation_test(setting)
exp.adaptation_varify(setting)

Model parameters:  18624
test 1905
output_len: 96
periodicity adaptation:0.9080523252487183
test 1905
periodicity adaptation verify:1.0128289461135864


### 数据分布适应性指标计算及验证

In [10]:
args.adaptation = True
args.periodicity = False
args.distribution = True
exp = Exp(args)
exp.adaptation_test(setting)
exp.adaptation_varify(setting)

Model parameters:  18624
test 1905
output_len: 96
distribution adaptation:1.0089056491851807
test 1905
distribution adaptation verify:0.9720181226730347


### 数据异常适应性指标计算及验证

In [11]:
args.adaptation = True
args.distribution = False
args.anomaly = True
exp = Exp(args)
exp.adaptation_test(setting)
exp.adaptation_varify(setting)

Model parameters:  18624
test 1905
output_len: 96
anomaly adaptation:1.326231837272644
test 1905
anomaly adaptation verify:1.110696792602539


### 对抗样本适应性指标计算

In [19]:
args.adaptation = True
args.anomaly = False
args.adversary = True
exp = Exp(args)
exp.adversarial_attack(setting)

test 1905


AttributeError: 'Args' object has no attribute 'output_attention'